In [8]:
#Download data from sentinel-2
#https://www.youtube.com/watch?v=D7Xjb28ckIU&ab_channel=krishnalodha

import os
from datetime import date, timedelta
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape

user = "tony96.donnangelo@gmail.com"
password = "-#gFcRBK+#hVFq2"

copernicus_user = user # copernicus User
copernicus_password = password # copernicus Password
ft = "POLYGON ((10.2667367495024 43.693479127517605, 10.2667367495024 42.897334215039905, 11.655473947599944 42.897334215039905, 11.655473947599944 43.693479127517605, 10.2667367495024 43.693479127517605))"  # Dimensione del ritaglio da geojson.io formato WKT
data_collection = "SENTINEL-2" # Sentinel satellite

today =  date.today()
today_string = today.strftime("%Y-%m-%d")
yesterday = today - timedelta(days=3)
yesterday_string = yesterday.strftime("%Y-%m-%d")



def get_keycloak(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }
    try:
        r = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Keycloak token creation failed. Reponse from the server was: {r.json()}"
        )
    return r.json()["access_token"]


json_ = requests.get(
    f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq '{data_collection}' and OData.CSC.Intersects(area=geography'SRID=4326;{ft}') and ContentDate/Start gt {yesterday_string}T00:00:00.000Z and ContentDate/Start lt {today_string}T00:00:00.000Z&$count=True&$top=1000"
).json()  
p = pd.DataFrame.from_dict(json_["value"]) # Fetch available dataset
if p.shape[0] > 0 :
    p["geometry"] = p["GeoFootprint"].apply(shape)
    productDF = gpd.GeoDataFrame(p).set_geometry("geometry") # Convert PD to GPD
    productDF = productDF[~productDF["Name"].str.contains("L1C")] # Remove L1C dataset
    print(f" total L2A tiles found {len(productDF)}")
    productDF["identifier"] = productDF["Name"].str.split(".").str[0]
    allfeat = len(productDF) 

    if allfeat == 0:
        print("No tiles found for today")
    else:
        ## download all tiles from server
        for index,feat in enumerate(productDF.iterfeatures()):
            try:
                session = requests.Session()
                keycloak_token = get_keycloak(copernicus_user,copernicus_password)
                session.headers.update({"Authorization": f"Bearer {keycloak_token}"})
                url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products({feat['properties']['Id']})/$value"
                response = session.get(url, allow_redirects=False)
                while response.status_code in (301, 302, 303, 307):
                    url = response.headers["Location"]
                    response = session.get(url, allow_redirects=False)
                print(feat["properties"]["Id"])
                file = session.get(url, verify=False, allow_redirects=True)

                with open(
                    f"{feat['properties']['identifier']}.zip", #location to save zip from copernicus 
                    "wb",
                ) as p:
                    print(feat["properties"]["Name"])
                    p.write(file.content)
            except:
                print("problem with server")
else :
    print('no data found')

 total L2A tiles found 7
problem with server
problem with server
problem with server
problem with server
5025d3ac-3449-4d5e-a708-811c9f453f96
S2A_MSIL2A_20240321T101721_N0510_R065_T32TPN_20240321T144051.SAFE
101f5c65-0f4b-4b1f-8876-fad03473d328
S2A_MSIL2A_20240321T101721_N0510_R065_T32TNP_20240321T144051.SAFE
83924370-c685-499c-908b-b2b4ec9689d0
S2A_MSIL2A_20240321T101721_N0510_R065_T32TNN_20240321T144051.SAFE


In [9]:
import zipfile
import matplotlib.pyplot as plt
import rasterio

# Path della cartella in cui sono stati scaricati i file zip
folder_path = "./"

# Estrai i file zip
for filename in os.listdir(folder_path):
    if filename.endswith(".zip"):
        with zipfile.ZipFile(os.path.join(folder_path, filename), "r") as zip_ref:
            zip_ref.extractall(folder_path)

# Visualizza le immagini
for filename in os.listdir(folder_path):
    if filename.endswith(".jp2"):  # Assumendo che le immagini siano in formato jp2
        with rasterio.open(os.path.join(folder_path, filename)) as src:
            img = src.read(1)  # Leggi il primo canale (banda) dell'immagine
            plt.imshow(img, cmap="gray")  # Visualizza l'immagine in scala di grigi
            plt.title(filename)  # Imposta il titolo dell'immagine
            plt.colorbar()  # Aggiungi una barra dei colori
            plt.show()  # Mostra l'immagine

BadZipFile: File is not a zip file